In [1]:
import findspark
findspark.init('/Users/dreyco676/spark-1.6.0-bin-hadoop2.6/')

import pyspark
from pyspark.sql import SQLContext
import langid
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import string
from pyspark.sql.types import *
import re
from nltk.corpus import stopwords
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from nltk import pos_tag, word_tokenize

sc = pyspark.SparkContext(appName="tweet_classifier")
sqlContext = SQLContext(sc)

In [2]:
# Load a text file and convert each line to a Row.
lines = sc.textFile("/Users/dreyco676/nlp_spark/data/classified_tweets.txt")
parts = lines.map(lambda l: l.split("\t"))
training = parts.map(lambda p: (p[0], p[1].strip()))

# The schema is encoded in a string.
schemaString = "tweet_text classification"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaTraining = sqlContext.createDataFrame(training, schema)

# Infer the schema, and register the DataFrame as a table.
schemaTraining = sqlContext.createDataFrame(training)
schemaTraining.registerTempTable("training")
named_df = schemaTraining.withColumnRenamed('_1', 'tweet').withColumnRenamed('_2', 'classification')
named_df.show(10)

+--------------------+--------------+
|               tweet|classification|
+--------------------+--------------+
|rt @jiffyclub: wi...|        python|
|rt @arnicas: ipyt...|        python|
|rt @treycausey: i...|        python|
|what's my best op...|        python|
|rt @raymondh: #py...|        python|
|ipython notebook ...|        python|
|checked out  for ...|        python|
|#openscience and ...|        python|
|"rt @ethanwhite: ...|        python|
|rt @python33r: th...|        python|
+--------------------+--------------+
only showing top 10 rows



In [7]:
# Use langid module to classify the language to make sure we are applying the correct cleanup actions
# https://github.com/saffsd/langid.py
def check_lang(data_str):
    predict_lang = langid.classify(data_str)
    if predict_lang[1] >= .9:
        language = predict_lang[0]
    else:
        language = 'NA'
    return language

check_lang_udf = udf(lambda x: check_lang(x), StringType())

In [3]:
def tag_and_remove(data_str):
    cleaned_str = ' '
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
    
    # break string into 'words'
    text = data_str.split()
    
    # tag the text and keep only those with the right tags
    tagged_text = pos_tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '
            
    return cleaned_str


tag_and_remove_udf = udf(lambda x: tag_and_remove(x), StringType())

In [4]:
def dummy_function(data_str):
    cleaned_str = 'dummyData' 
    return cleaned_str


dummy_function_udf = udf(dummy_function, StringType())

In [13]:
df = named_df.withColumn("dummy", tag_and_remove_udf(named_df['tweet']))
df.show(5)

Py4JJavaError: An error occurred while calling o285.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/dreyco676/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/dreyco676/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/dreyco676/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-7-48bcc12eb1c8>", line 4, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.py